In [1]:
import time
from datetime import date

import numpy as np
import pandas as pd

import pandas_datareader as pdr
import yfinance as yf

import matplotlib.pyplot as plt

import requests

from io import StringIO

In [2]:
pd.set_option("display.max_rows", None) # show full of showing rows
pd.set_option("display.max_columns", None) # show full of showing cols
pd.set_option("display.max_colwidth", None) # show full width of showing cols

In [3]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

# 1. IPO Filings Web Scraping and Data Processing

What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)

In [4]:
URL = "https://stockanalysis.com/ipos/filings/"

response = requests.get(URL, headers=HEADERS)
df_ipo = pd.read_html(StringIO(response.text))[0]
df_ipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     324 non-null    object
 1   Symbol          324 non-null    object
 2   Company Name    324 non-null    object
 3   Price Range     324 non-null    object
 4   Shares Offered  324 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [5]:
df_ipo.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 3, 2024",TBN,Tamboran Resources Corporation,-,-
1,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
4,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-


In [6]:
df_ipo["Filing Date"] = pd.to_datetime(df_ipo["Filing Date"], format="%b %d, %Y")

In [7]:
df_ipo.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-


In [8]:
df_ipo.replace({"-" : np.nan}, inplace=True)

In [9]:
df_ipo["Shares Offered"] = df_ipo["Shares Offered"].astype("float64")

In [10]:
df_ipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     324 non-null    datetime64[ns]
 1   Symbol          324 non-null    object        
 2   Company Name    324 non-null    object        
 3   Price Range     256 non-null    object        
 4   Shares Offered  250 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 12.8+ KB


In [11]:
df_ipo.head(12)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,NaN,NaN
5,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN
6,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,NaN
7,2024-04-15,GAUZ,Gauzy Ltd.,NaN,NaN
8,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,NaN,NaN
9,2024-04-05,SPHL,Springview Holdings Ltd,$4.00 - $5.00,2000000.0


In [12]:
def avg_price(value) -> float:
    
    if isinstance(value, str):
        if "-" in value:
            lower, higher = value.split(" - ")
            return (float(lower.replace("$", "")) + float(higher.replace("$", ""))) / 2
        elif "$" in value:
            return value.split("$")[1]
    else:
        return np.nan

In [13]:
df_ipo["Avg Price"] = df_ipo["Price Range"].apply(lambda x: avg_price(x))
df_ipo["Avg Price"] = df_ipo["Avg Price"].astype("float64")

In [14]:
df_ipo.head(20)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg Price
0,2024-05-03,TBN,Tamboran Resources Corporation,NaN,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00
4,2024-04-26,HDL,Super Hi International Holding Ltd.,NaN,NaN,NaN
5,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,5.00
6,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,NaN,5.00
7,2024-04-15,GAUZ,Gauzy Ltd.,NaN,NaN,NaN
8,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,NaN,NaN,NaN
9,2024-04-05,SPHL,Springview Holdings Ltd,$4.00 - $5.00,2000000.0,4.50


In [15]:
df_ipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324 entries, 0 to 323
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Filing Date     324 non-null    datetime64[ns]
 1   Symbol          324 non-null    object        
 2   Company Name    324 non-null    object        
 3   Price Range     256 non-null    object        
 4   Shares Offered  250 non-null    float64       
 5   Avg Price       256 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 15.3+ KB


In [16]:
df_ipo["Shares Offered Value"] = df_ipo["Shares Offered"] * df_ipo["Avg Price"]

In [17]:
df_ipo.head(20)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg Price,Shares Offered Value
0,2024-05-03,TBN,Tamboran Resources Corporation,NaN,NaN,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,NaN,NaN,NaN,NaN
5,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,5.00,NaN
6,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,NaN,5.00,NaN
7,2024-04-15,GAUZ,Gauzy Ltd.,NaN,NaN,NaN,NaN
8,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,NaN,NaN,NaN,NaN
9,2024-04-05,SPHL,Springview Holdings Ltd,$4.00 - $5.00,2000000.0,4.50,9000000.0


In [18]:
df_ipo[(df_ipo["Filing Date"].dt.year == 2023) & (df_ipo["Filing Date"].dt.dayofweek == 4)]["Shares Offered Value"].shape

(32,)

In [19]:
df_ipo[(df_ipo["Filing Date"].dt.year == 2023) & (df_ipo["Filing Date"].dt.dayofweek == 4)]["Shares Offered Value"].notna().sum()

25

In [20]:
print(f"$ {round(df_ipo[(df_ipo['Filing Date'].dt.year == 2023) & (df_ipo['Filing Date'].dt.dayofweek == 4)]['Shares Offered Value'].sum()/1e6)} millions")

$ 286 millions


# 2. IPOs "Fixed days hold" strategy

Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this link. Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:
- You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
- You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.

Additional:
- You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
- What's your recommendation: Do you suggest pursuing this strategy for an optimal X?

In [96]:
URL = "https://stockanalysis.com/ipos/2023/"

response = requests.get(URL, headers=HEADERS)
df_ipos_2023 = pd.read_html(StringIO(response.text))[0]
df_ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [97]:
URL = "https://stockanalysis.com/ipos/2024/"

response = requests.get(URL, headers=HEADERS)
df_ipos_2024 = pd.read_html(StringIO(response.text))[0]
df_ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      67 non-null     object
 1   Symbol        67 non-null     object
 2   Company Name  67 non-null     object
 3   IPO Price     67 non-null     object
 4   Current       67 non-null     object
 5   Return        67 non-null     object
dtypes: object(6)
memory usage: 3.3+ KB


In [98]:
df_ipos_2024.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 9, 2024",SVCO,"Silvaco Group, Inc.",$19.00,$19.80,7.68%
1,"May 9, 2024",PAL,"Proficient Auto Logistics, Inc.",$15.00,$14.75,-0.90%
2,"May 8, 2024",NNE,Nano Nuclear Energy Inc.,$4.00,$5.05,34.75%
3,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.50,18.75%
4,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.17,-20.75%


In [99]:
df_ipos = pd.concat([df_ipos_2023, df_ipos_2024], ignore_index=True)

In [100]:
df_ipos["IPO Date"] = pd.to_datetime(df_ipos["IPO Date"], format="%b %d, %Y")

In [101]:
df_ipos.head(20)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.84,-29.00%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.61,-18.65%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.63,-84.05%
5,2023-12-04,AFJK,"Aimei Health Technology Co., Ltd Ordinary Share",$10.00,$10.30,3.00%
6,2023-12-01,GSIW,Garden Stage Limited,$4.00,$7.04,76.00%
7,2023-11-30,FEBO,Fenbo Holdings Limited,$5.00,$12.61,152.10%
8,2023-11-21,CLBR,Colombier Acquisition Corp.,$10.00,$10.34,3.40%
9,2023-11-21,ELAB,"Elevai Labs, Inc.",$4.00,$0.68,-83.13%


In [102]:
df_ipos["Symbol"] = df_ipos["Symbol"].str.replace("PTHR", "PTHRU")

In [103]:
df_ipos[(df_ipos["IPO Date"] < "2024-03-01") & (df_ipos["Symbol"] != "RYZB")].count()

IPO Date        184
Symbol          184
Company Name    184
IPO Price       184
Current         184
Return          184
dtype: int64

In [104]:
df_ipos = df_ipos[(df_ipos["IPO Date"] < "2024-03-01") & (df_ipos["Symbol"] != "RYZB")]

In [105]:
# df_ipos.replace({"-" : np.nan}, inplace=True)

In [106]:
df_ipos.head(20)

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.84,-29.00%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.61,-18.65%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.63,-84.05%
5,2023-12-04,AFJK,"Aimei Health Technology Co., Ltd Ordinary Share",$10.00,$10.30,3.00%
6,2023-12-01,GSIW,Garden Stage Limited,$4.00,$7.04,76.00%
7,2023-11-30,FEBO,Fenbo Holdings Limited,$5.00,$12.61,152.10%
8,2023-11-21,CLBR,Colombier Acquisition Corp.,$10.00,$10.34,3.40%
9,2023-11-21,ELAB,"Elevai Labs, Inc.",$4.00,$0.68,-83.13%


In [107]:
df_ipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 220
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      184 non-null    datetime64[ns]
 1   Symbol        184 non-null    object        
 2   Company Name  184 non-null    object        
 3   IPO Price     184 non-null    object        
 4   Current       184 non-null    object        
 5   Return        184 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 10.1+ KB


In [108]:
df_ipos["IPO Price"] = df_ipos["IPO Price"].str.replace("$", "").astype("float64")
df_ipos["Current"] = df_ipos["Current"].str.replace("$", "").astype("float64")
df_ipos["Return"] = df_ipos["Return"].str.replace("%", "").astype("float64")

In [109]:
df_ipos["Return"] = df_ipos["Return"] / 100

In [110]:
df_ipos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 220
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      184 non-null    datetime64[ns]
 1   Symbol        184 non-null    object        
 2   Company Name  184 non-null    object        
 3   IPO Price     184 non-null    float64       
 4   Current       184 non-null    float64       
 5   Return        184 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 10.1+ KB


In [111]:
df_ipos.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,10.0,10.05,0.0050
1,2023-12-19,LGCB,Linkage Global Inc,4.0,2.84,-0.2900
2,2023-12-15,ZKH,ZKH Group Limited,15.5,12.61,-0.1865
3,2023-12-15,BAYA,Bayview Acquisition Corp,10.0,10.18,0.0180
4,2023-12-14,INHD,Inno Holdings Inc.,4.0,0.63,-0.8405


In [121]:
symbols = df_ipos["Symbol"].to_list()
df_tickers = []
for ticker in symbols:
    try:
        start = df_ipos[df_ipos["Symbol"] == ticker]["IPO Date"].dt.strftime("%Y-%m-%d").values[0]
        df_ticker = yf.download(tickers=ticker, start=start, interval="1d")[:31]
        ticker_growth_values = {i : (df_ticker["Adj Close"].shift(-i) / df_ticker["Adj Close"]).iloc[0] for i in range(1, 31)}
        df_ticker_growth = pd.DataFrame(ticker_growth_values, index=[ticker])
        df_tickers.append(df_ticker_growth)
    except Exception as e:
        print(f"Error getting data for {symbol}: {str(e)}")
        
df_tickers = pd.concat(df_tickers)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [122]:
df_tickers_describe = df_tickers.describe()
df_tickers_describe[df_tickers_describe.index == "75%"].T.sort_values(by="75%", ascending=False)[:5]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
count,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,182.000000
mean,0.952830,0.942563,0.936669,0.928229,0.914873,0.911443,0.900442,0.898945,0.903139,0.897291,0.909139,0.932955,0.934011,0.933149,0.926962,0.918212,0.911927,0.899323,0.904906,0.917839,0.920504,0.917932,0.917840,0.912839,0.907297,0.906816,0.893045,0.898985,0.895944,0.889614
std,0.163292,0.197248,0.239528,0.255566,0.287638,0.301267,0.292431,0.314881,0.346765,0.360332,0.409612,0.595000,0.540131,0.572437,0.499456,0.437747,0.408700,0.427723,0.441352,0.486957,0.505442,0.480742,0.494000,0.489163,0.492651,0.536092,0.501243,0.500254,0.479260,0.452568
min,0.153569,0.108733,0.086641,0.094257,0.081118,0.087677,0.085261,0.090123,0.095616,0.096997,0.081118,0.078357,0.077321,0.073870,0.064895,0.056610,0.056265,0.056921,0.053331,0.049879,0.048326,0.048326,0.049361,0.048326,0.048326,0.045219,0.044529,0.046945,0.042803,0.040387
25%,0.930659,0.908365,0.872044,0.861604,0.797155,0.799570,0.796129,0.774288,0.755546,0.712461,0.680322,0.702877,0.726835,0.736063,0.745211,0.733934,0.735589,0.700431,0.684833,0.654484,0.678704,0.622479,0.633110,0.641305,0.635149,0.600923,0.615346,0.646588,0.629096,0.623529
50%,1.000000,1.000000,0.998782,0.995516,0.993903,0.970152,0.974645,0.980808,0.985234,0.978063,0.970734,0.982117,0.985658,0.989106,0.979026,0.991504,0.990762,0.961719,0.972635,0.977083,0.984326,0.994409,0.991045,0.987043,0.972843,0.977839,0.988182,0.975463,0.979346,0.977535
75%,1.017014,1.019670,1.011578,1.013020,1.008295,1.006900,1.006871,1.006941,1.011545,1.007391,1.011155,1.016016,1.018676,1.019706,1.016284,1.014190,1.011330,1.014949,1.016668,1.015145,1.014209,1.035524,1.028015,1.035345,1.026972,1.028256,1.036537,1.039693,1.026360,1.023718
max,1.362069,1.464015,2.380000,2.083710,2.262443,2.529870,2.173913,2.359740,2.751948,3.176087,3.591305,6.779545,5.622727,6.406818,4.656818,4.072398,3.212670,3.500000,3.622727,3.970454,4.500000,3.871041,3.846154,3.803394,3.427273,4.817886,4.434146,4.570732,4.247155,3.629268
